## **Fine-Tuning SLMs & LLMs on SageMaker**

Fine-tuning helps adapt pre-trained models (like BERT, RoBERTa, DistilBERT, TinyBERT) to specific tasks such as sentiment analysis, topic classification, or entity recognition.

### **1. Process Overview**
1. **Choose a Pre-trained Model**  
   - Select an SLM (e.g., DistilBERT, TinyBERT) or LLM (e.g., BERT, RoBERTa) from **Hugging Face Model Hub** or **SageMaker JumpStart**.
   
2. **Prepare Data**  
   - Format data in **CSV, JSON, or Parquet**.
   - Tokenize using `transformers` library.

3. **Set Up SageMaker Environment**  
   - Define an **Amazon SageMaker Training Job**.
   - Use SageMaker’s **Hugging Face Estimator**.

4. **Fine-Tune the Model**  
   - Define hyperparameters.
   - Use distributed training if necessary.

5. **Evaluate & Deploy**  
   - Deploy using **SageMaker Endpoint**.
   - Evaluate on test data.

---

## **2. Code Snippet for Fine-Tuning SLM/LLM on SageMaker**

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker import get_execution_role

# Initialize SageMaker session & role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define the Hugging Face model (BERT, RoBERTa, DistilBERT)
hyperparameters = {
    "epochs": 3,
    "train_batch_size": 16,
    "model_name": "distilbert-base-uncased",  # Change for LLMs like BERT, RoBERTa
}

# Define training script location
train_script = "train.py"  # Custom training script

# Set up SageMaker Estimator
huggingface_estimator = HuggingFace(
    entry_point=train_script,
    source_dir="./",
    instance_type="ml.p3.2xlarge",  # Use ml.g5 for large models
    instance_count=1,
    role=role,
    transformers_version="4.17",
    pytorch_version="1.10",
    py_version="py38",
    hyperparameters=hyperparameters,
)

# Start training job
huggingface_estimator.fit({"train": "s3://your-bucket/train-data/"})

## **3. Deployment & Inference**
Once training is complete, we can deploy the model as an API using a **SageMaker endpoint**.

In [ ]:
# Deploy trained model
predictor = huggingface_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"  # Change for large models
)

# Run inference
input_text = {"inputs": "What is the sentiment of this sentence?"}
response = predictor.predict(input_text)
print(response)
